# Library 설치

In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options

from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
from tqdm import tqdm
import math
import os, json, time, html, re, random, csv
from datetime import datetime
from urllib.parse import urlparse, parse_qs, unquote
from typing import Dict, List, Optional

# 1. 드라이버 초기화

In [3]:
# --- 1. 드라이버 초기화 (로컬 Windows, 내 프로필 미사용) ---
# 소수점 아래 2자리까지 표시하도록 설정
pd.options.display.float_format = '{:.2f}'.format

def create_driver(headless=False, remote_debug_port=9222):
    opts = Options()
    # ✅ 안정화 필수 옵션 (Windows에서도 유효)
    opts.add_argument("--no-sandbox")
    opts.add_argument("--disable-dev-shm-usage")
    opts.add_argument("--disable-gpu")
    opts.add_argument(f"--remote-debugging-port={remote_debug_port}")
    opts.add_argument("--lang=ko_KR")
    opts.add_argument("window-size=1920x1080")
    # UA가 필요하면 아래 주석 해제
    # opts.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/140.0.0.0 Safari/537.36")

    # ❌ 내 크롬 프로필 사용 금지 (충돌/잠금 방지)
    # opts.add_argument(f"user-data-dir=...")  # 사용하지 않음

    if headless:
        # 최신 크롬은 headless=new 권장
        opts.add_argument("--headless=new")

    # 자동화 흔적 최소화
    opts.add_experimental_option("excludeSwitches", ["enable-automation"])
    opts.add_experimental_option("useAutomationExtension", False)

    service = Service(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=service, options=opts)
    try:
        driver.execute_script("Object.defineProperty(navigator, 'webdriver', {get: () => undefined})")
    except Exception:
        pass
    return driver

try:
    driver = create_driver(headless=False, remote_debug_port=9222)
except Exception as e:
    print(f"드라이버 초기화 중 오류 발생: {e}")
    raise


# 2. 검색 페이지 이동

In [4]:
# --- 2. 검색 페이지 이동 ---
search_url = "https://search.shopping.naver.com/" # 예시 URL
driver.get(search_url)
# 브라우저에 검색어 직접 입력해서 검색

# 3. 상품 목록 수집

In [5]:
# --- 3. 상품 목록 수집 (무한 스크롤 고려 + 셀렉터 최신화 + 할인율 소수) ---

# ===== Step3 함수 설정 =====

# 부드러운 스크롤 함수
def smooth_scroll_by(driver, scroll_by_y, duration):
    """
    주어진 픽셀(scroll_by_y)만큼 주어진 시간(duration) 동안 부드럽게 스크롤합니다.
    """
    start_time = time.time()
    current_y = driver.execute_script("return window.scrollY")
    target_y = current_y + scroll_by_y

    while time.time() < start_time + duration:
        # 경과 시간을 기반으로 진행률 계산 (0.0 to 1.0)
        progress = (time.time() - start_time) / duration
        
        # 진행률이 1을 넘지 않도록 보정
        progress = min(progress, 1.0)

        # Ease-in-out 효과: 시작과 끝을 부드럽게 만듭니다.
        # sin 함수를 이용해 S자 곡선을 그려 가속/감속 효과를 냅니다.
        eased_progress = 0.5 * (1 - math.cos(progress * math.pi))
        
        # 현재 스크롤 위치 계산
        scroll_to = current_y + (scroll_by_y * eased_progress)
        driver.execute_script(f"window.scrollTo(0, {scroll_to})")
        
        # 브라우저가 렌더링할 시간을 주기 위한 짧은 대기
        time.sleep(0.01)

    # 오차 보정을 위해 스크롤이 끝나면 목표 지점으로 정확히 이동
    driver.execute_script(f"window.scrollTo(0, {target_y})")

def _step3_guess_key_col(cols):
    if "상품링크" in cols:
        return "상품링크"
    for c in cols:
        lc = str(c).lower()
        if ("url" in lc) or ("link" in lc) or ("링크" in c):
            return c
    return None

def _step3_make_key(rec: dict, fallback=None):
    # 1) 상품링크 최우선
    if rec.get("상품링크"):
        return str(rec["상품링크"]).strip()
    # 2) 후보 순회
    for k in _STEP3_KEY_COLS:
        if rec.get(k):
            return str(rec[k]).strip()
    # 3) fallback
    if fallback and rec.get(fallback):
        return str(rec[fallback]).strip()
    # 4) 마지막 수단: 제목
    if rec.get("제목"):
        return str(rec["제목"]).strip()
    return None

def _step3_load_ckpt():
    """체크포인트 CSV를 읽어 (rows, key_set, key_col) 반환. 없으면 빈 구조."""
    if not os.path.exists(STEP3_CKPT_CSV):
        return [], set(), None
    df = pd.read_csv(STEP3_CKPT_CSV)
    # 필수 컬럼 보정
    for col in ["상품링크", "제목", "원래가격", "현재가격", "할인율", "스토어"]:
        if col not in df.columns:
            df[col] = None
    key_col = _step3_guess_key_col(df.columns)
    # 키셋
    keys = set()
    if key_col:
        keys |= set(df[key_col].dropna().astype(str).str.strip().tolist())
    else:
        # 키컬럼이 없으면 제목으로라도
        if "제목" in df.columns:
            keys |= set(df["제목"].dropna().astype(str).str.strip().tolist())
    rows = df.to_dict("records")
    print(f"[Step3] 체크포인트 로드: {len(rows)}행 (key_col={key_col or '없음'})")
    return rows, keys, key_col

def _step3_append_ckpt_row(rec: dict):
    """1건씩 즉시 append 저장(헤더 자동)."""
    write_header = not os.path.exists(STEP3_CKPT_CSV)
    with open(STEP3_CKPT_CSV, "a", newline="", encoding="utf-8-sig") as f:
        writer = csv.DictWriter(f, fieldnames=list(rec.keys()))
        if write_header:
            writer.writeheader()
        writer.writerow(rec)

def extract_smartstore_link_from_li(li):
    """
    네이버 플러스스토어 목록의 각 상품 li에서 상세페이지 링크(href) 추출.
    규칙: div.basicProductCard_basic_product_card__TdrHT 아래의
          a.basicProductCard_link__urzND[href] 가 상세 링크.
    """
    # 1) 카드 컨테이너 찾기
    container = None
    try:
        container = li.find_element(
            By.CSS_SELECTOR,
            'div.basicProductCard_basic_product_card__TdrHT'
        )
    except:
        # 클래스가 바뀐 경우 대비해 li 자체를 fallback으로 사용
        container = li

    # 2) 대표 링크 우선 추출
    try:
        a = container.find_element(
            By.CSS_SELECTOR, 'a.basicProductCard_link__urzND[href]'
        )
        href = a.get_attribute('href')
        if href:
            return href
    except:
        pass

    # 3) 보강: data-shp-contents-dtl 안의 click_url 또는 임의의 a[href]
    candidates = []
    try:
        candidates = container.find_elements(By.CSS_SELECTOR, 'a[href]')
    except:
        candidates = []

    for a in candidates:
        # data-shp-contents-dtl → click_url 시도
        try:
            meta = a.get_attribute("data-shp-contents-dtl")
            if meta:
                meta_json = json.loads(html.unescape(meta))
                for entry in meta_json:
                    if entry.get("key") == "click_url" and entry.get("value"):
                        return entry.get("value")
        except:
            pass
        # 일반 href
        try:
            href = a.get_attribute("href")
            if href:
                return href
        except:
            pass

    return None

def to_int(text):
    if not text:
        return None
    digits = re.sub(r"[^\d]", "", text)
    return int(digits) if digits else None

def parse_discount_fraction(text):
    # '34%', '34 % 할인' → 0.34, 없으면 None
    if not text:
        return None
    m = re.search(r"(\d+)", text)
    return (int(m.group(1)) / 100.0) if m else None

def extract_click_url_from(elem):
    # mall_link data JSON → href → 상단 링크 순
    anchor_selectors = [
        'a[class^="productCardMallLink_mall_link__"]',
        'a[class^="basicProductCard_link__"]',
    ]
    for sel in anchor_selectors:
        try:
            a = elem.find_element(By.CSS_SELECTOR, sel)
        except:
            continue
        try:
            meta = a.get_attribute("data-shp-contents-dtl")
            if meta:
                meta_json = json.loads(html.unescape(meta))
                for entry in meta_json:
                    if entry.get("key") == "click_url":
                        return entry.get("value")
        except:
            pass
        try:
            href = a.get_attribute("href")
            if href:
                return href
        except:
            pass
    return None

def collect_visible_cards(processed_keys, rows_out):
    """현재 화면 근방에 존재하는 카드만 수집. 이미 본 항목은 건너뜀."""
    cards = driver.find_elements(By.CSS_SELECTOR, card_selector)
    new_added = 0
    for li in cards:
        # 제목
        try:
            title = li.find_element(By.CSS_SELECTOR, title_sel).text.strip()
        except:
            continue

        # 링크 (키)
        link = extract_smartstore_link_from_li(li)
        key  = link or title  # 링크가 없으면 제목으로 임시 키

        if key in processed_keys:
            continue

        # 스토어
        try:
            store = li.find_element(By.CSS_SELECTOR, store_sel).text.strip()
        except:
            store = None

        # 가격들
        try:
            current_text = li.find_element(By.CSS_SELECTOR, price_sel).text
        except:
            current_text = None
        try:
            original_text = li.find_element(By.CSS_SELECTOR, orig_sel).text
        except:
            original_text = None
        try:
            discount_text = li.find_element(By.CSS_SELECTOR, disc_sel).text
        except:
            discount_text = None

        current_price  = to_int(current_text)
        original_price = to_int(original_text)
        discount_frac  = parse_discount_fraction(discount_text)  # 0.34 형태

        rec = {
            "스토어": store,
            "제목": title,
            "원래가격": original_price,
            "현재가격": current_price,
            "할인율": discount_frac,
            "상품링크": link,
        }

        # 체크포인트 기준 중복도 함께 고려
        ck_key = _step3_make_key(rec, None)
        if ck_key and ck_key in processed_keys:
            continue  # 이미 체크포인트에 존재 → SKIP

        # 신규 → 즉시 체크포인트에 1건 append 저장
        try:
            _step3_append_ckpt_row(rec)
        except Exception as _e:
            print("체크포인트 append 저장 실패(무시):", _e)

        # 메모리/세션 중복 방지용
        rows_out.append(rec)
        processed_keys.add(ck_key or key)
        new_added += 1
    return new_added, len(cards)

## 수집 메인 코드

In [7]:
# ===== Step3 메인 코드 시작 =====

wait = WebDriverWait(driver, 20)

# ===== 체크포인트 설정 (Step3 전용) =====
STEP3_CKPT_DIR = "step3_checkpoint"
STEP3_CKPT_CSV = os.path.join(STEP3_CKPT_DIR, "step3_ckeckpoint.csv")  # 요청 그대로 철자 유지

os.makedirs(STEP3_CKPT_DIR, exist_ok=True)

# 키 후보: 중복 판별 기준(링크 우선)
_STEP3_KEY_COLS = ["상품링크", "url", "link", "상품URL", "상품_link", "상품_링크", "상세링크"]


# 결과 영역 등장 대기
wait.until(EC.presence_of_element_located(
    (By.CSS_SELECTOR, 'ul[class^="compositeCardList_product_list__"]')
))

# ===== 유틸들 =====
card_selector = 'ul[class^="compositeCardList_product_list__"] > li[class^="compositeCardContainer_composite_card_container__"]'
title_sel    = 'strong[class^="productCardTitle_product_card_title__"]'
store_sel    = 'a[class^="productCardMallLink_mall_link__"] span[class^="productCardMallLink_mall_name__"]'
price_sel    = 'span[class^="priceTag_price__"]'
orig_sel     = 'span[class^="priceTag_original_price__"]'
disc_sel     = 'span[class^="priceTag_discount_ratio__"]'


# ===== 메인 루프: 스크롤 ↔ 수집 반복 =====
# 재개(Resume): 체크포인트에서 읽어오기
rows, processed, ck_key_col = _step3_load_ckpt()
rounds = 0  # 라운드 카운트는 새로 시작(원하면 progress.json 같은 메타를 다시 넣어도 됨)

stable_needed = random.randint(3, 5)  # 연속 정지 임계
stable_count  = 0

prev_height = driver.execute_script("return document.body.scrollHeight")
prev_total  = len(rows)  # 누적 수집 개수(재개 시 반영)
prev_domcnt = 0

start_time  = time.time()
MAX_ROUNDS  = 500         # 최대 500 라운드
MAX_SECONDS = 3600        # 최대 60분

try:
    while True:
        # 1) 현재 보이는 카드 수집
        added, domcnt = collect_visible_cards(processed, rows)

        # 2) 스크롤 (뷰포트 × 1.5–2.0, 부드럽게)
        vh = driver.execute_script("return Math.max(window.innerHeight, document.documentElement.clientHeight);")
        factor = random.uniform(1.5, 2.0)                # 권장 범위: 1.5–2.0
        scroll_distance = int(vh * factor)

        # 기존 감각 유지: 거리 비례 시간(대략 1000–1600 px/s)
        px_per_sec = random.uniform(1000, 1600)
        scroll_duration = max(1.2, min(4.0, scroll_distance / px_per_sec))

        print(f"[round {rounds+1}] 수집 + 스크롤 {scroll_distance}px / {scroll_duration:.1f}s | 새 {added}개 | 누적 {len(rows)}개")
        smooth_scroll_by(driver, scroll_distance, scroll_duration)


        # 3) 로딩 대기
        time.sleep(random.uniform(3.0, 5.0))

        # 4) 증가 여부 측정
        new_height = driver.execute_script("return document.body.scrollHeight")
        grew_height = new_height > prev_height
        grew_total  = len(rows) > prev_total
        grew_dom    = domcnt > prev_domcnt

        if not grew_height and not grew_total and not grew_dom:
            stable_count += 1
            print(f"  ↳ 증가 없음 → 연속 정지 {stable_count}/{stable_needed}")
            if stable_count >= stable_needed:
                print(f"끝까지 로드됨: 총 {len(rows)}개 수집, 라운드 {rounds+1}회")
                break
        else:
            stable_count = 0
            prev_height  = max(prev_height, new_height)
            prev_total   = len(rows)
            prev_domcnt  = domcnt

        rounds += 1
        if rounds >= MAX_ROUNDS:
            print("안전장치: 최대 라운드 도달 → 종료")
            break
        if time.time() - start_time > MAX_SECONDS:
            print("안전장치: 최대 시간 도달 → 종료")
            break

    # 마지막으로 화면에 남아있는 카드도 한 번 더 수집
    added, _ = collect_visible_cards(processed, rows)
    if added:
        print(f"마지막 추가 수집 {added}개 반영 (최종 {len(rows)}개)")

finally:
    # 최종본은 체크포인트 CSV 기준으로 dedupe하여 저장(안전)
    if os.path.exists(STEP3_CKPT_CSV):
        df_ck = pd.read_csv(STEP3_CKPT_CSV)
        kcol = _step3_guess_key_col(df_ck.columns) or ("제목" if "제목" in df_ck.columns else None)
        if kcol:
            df_ck = df_ck.drop_duplicates(subset=[kcol], keep="last")
        df_ck.to_csv("step3_products.csv", index=False, encoding="utf-8-sig")
        print("[Step3] 최종 병합본 저장:", len(df_ck))
    else:
        # 체크포인트 없이 rows만 있을 수 있음(첫 실행 바로 종료 등)
        df_tmp = pd.DataFrame(rows)
        df_tmp.to_csv("step3_products.csv", index=False, encoding="utf-8-sig")
        print("[Step3] 최종 저장(메모리 rows 기준):", len(df_tmp))


df_list = pd.DataFrame(rows)
print(f"최종 수집 상품 수: {len(df_list)}")

[Step3] 체크포인트 로드: 6695행 (key_col=상품링크)
[round 1] 수집 + 스크롤 2379px / 1.8s | 새 0개 | 누적 6695개
[round 2] 수집 + 스크롤 2409px / 2.3s | 새 50개 | 누적 6745개
[round 3] 수집 + 스크롤 2573px / 2.0s | 새 0개 | 누적 6745개
[round 4] 수집 + 스크롤 2469px / 2.1s | 새 50개 | 누적 6795개
[round 5] 수집 + 스크롤 2464px / 2.3s | 새 50개 | 누적 6845개
[round 6] 수집 + 스크롤 2480px / 1.9s | 새 0개 | 누적 6845개
[round 7] 수집 + 스크롤 2015px / 1.5s | 새 50개 | 누적 6895개
[round 8] 수집 + 스크롤 2535px / 1.6s | 새 0개 | 누적 6895개
[round 9] 수집 + 스크롤 2357px / 1.6s | 새 50개 | 누적 6945개
[round 10] 수집 + 스크롤 2079px / 1.7s | 새 50개 | 누적 6995개
[round 11] 수집 + 스크롤 2466px / 1.7s | 새 0개 | 누적 6995개
[round 12] 수집 + 스크롤 2207px / 1.6s | 새 50개 | 누적 7045개
[round 13] 수집 + 스크롤 2413px / 2.0s | 새 0개 | 누적 7045개
[round 14] 수집 + 스크롤 2388px / 2.4s | 새 50개 | 누적 7095개
[round 15] 수집 + 스크롤 2495px / 2.0s | 새 0개 | 누적 7095개
[round 16] 수집 + 스크롤 2335px / 1.6s | 새 50개 | 누적 7145개
[round 17] 수집 + 스크롤 2245px / 2.0s | 새 50개 | 누적 7195개
[round 18] 수집 + 스크롤 2292px / 2.1s | 새 0개 | 누적 7195개
[round 19] 수집 + 스크롤 2147

In [8]:
df_list.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10095 entries, 0 to 10094
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   스토어     10095 non-null  object 
 1   제목      10095 non-null  object 
 2   원래가격    5281 non-null   float64
 3   현재가격    10095 non-null  int64  
 4   할인율     5068 non-null   float64
 5   상품링크    10095 non-null  object 
dtypes: float64(2), int64(1), object(3)
memory usage: 473.3+ KB


In [9]:
df_list.head()

,스토어,제목,원래가격,현재가격,할인율,상품링크
0,바이브랩,"바이브랩 4주 솔루션 초록 탈모 샴푸 우디플로럴머스크, 500ml, 1개",32000.00,30900,0.03,https://ader.naver.com/v1/9xKZUaGVHVRpl6-BM9xf...
1,바이브랩,"바이브랩 4주 솔루션 초록 탈모 샴푸 우디플로럴머스크, 500ml, 2개",62000.00,60900,0.01,https://ader.naver.com/v1/BKdxaZ_OPKeK4lK-ly5Z...
2,Solep,두피진정 탈모샴푸300ml+100ml 앰플증정,35000.00,25400,0.27,https://ader.naver.com/v1/XqrZw161cHIqOpjmniEy...
3,아모레퍼시픽몰 헤어바디,라보에이치 여름 쿨샴푸 지성 탈모 대용량 두피스케일링 400ml&400ml리필&180ml,44600.00,29800,0.33,https://ader.naver.com/v1/BKdxaZ_OPKeK4lK-ly5Z...
4,아모레퍼시픽몰 헤어바디,려 루트젠 두피 에센스 대용량 145ml 두피 영양제 여성 남성 탈모 앰플,36000.00,23900,0.33,https://smartstore.naver.com/main/products/115...


In [10]:
df_list.tail()

,스토어,제목,원래가격,현재가격,할인율,상품링크
10090,내일배송샵,레시피 선몬랩 리프레시 쿨링 스칼프 스프레이 70ml 1065050,NaN,6790,NaN,https://smartstore.naver.com/main/products/118...
10091,sell48,히피펌 앞머리 화이트 세팅펌 헤어 볼륨 소형 드라이기 흰색,47400.00,23700,0.50,https://smartstore.naver.com/main/products/123...
10092,스마일핫픽,닥터포헤어 폴리젠 바이오3 샴푸 750ml+100mlx2개,44000.00,42900,0.02,https://smartstore.naver.com/main/products/121...
10093,베레모던,멀티 500ml 헤어 머리 스타일링 미용실 펌제 파마약,28520.00,28230,0.01,https://smartstore.naver.com/main/products/119...
10094,블루디마켓,지청 헤어케어 자양 520ml 어성초 샴푸액 함유,36290.00,35920,0.01,https://smartstore.naver.com/main/products/118...


In [11]:
# -- 전처리 --
# 제목 앞에 '해외' 붙은 것 -> [해외]로 변경
df_list['제목'] = df_list['제목'].str.replace(r'^해외', '[해외] ', regex=True)
df_list.tail()

,스토어,제목,원래가격,현재가격,할인율,상품링크
10090,내일배송샵,레시피 선몬랩 리프레시 쿨링 스칼프 스프레이 70ml 1065050,NaN,6790,NaN,https://smartstore.naver.com/main/products/118...
10091,sell48,히피펌 앞머리 화이트 세팅펌 헤어 볼륨 소형 드라이기 흰색,47400.00,23700,0.50,https://smartstore.naver.com/main/products/123...
10092,스마일핫픽,닥터포헤어 폴리젠 바이오3 샴푸 750ml+100mlx2개,44000.00,42900,0.02,https://smartstore.naver.com/main/products/121...
10093,베레모던,멀티 500ml 헤어 머리 스타일링 미용실 펌제 파마약,28520.00,28230,0.01,https://smartstore.naver.com/main/products/119...
10094,블루디마켓,지청 헤어케어 자양 520ml 어성초 샴푸액 함유,36290.00,35920,0.01,https://smartstore.naver.com/main/products/118...


# 4. 상세 페이지 스펙 수집

In [59]:
# --- 4. 상세 페이지 스펙 수집 (각 링크 방문 → 1.5화면 스크롤 → 표 파싱) ---

# === df_list 자동 로더 (Step3 산출물에서 복원) ===

# 찾을 후보들(순서대로 시도)
_STEP3_SOURCES = [
    # Step3 체크포인트(요청 명칭)
    "step3_checkpoint/step3_ckeckpoint.csv",
    # Step3 최종 저장본(있다면)
    "step3_products.csv",
    # (옵션) 다른 폴더를 쓰신 적이 있다면 여기에 추가
    "checkpoints_step3/step3_products_checkpoint.csv",
]

def _guess_url_col(df: pd.DataFrame) -> str | None:
    # '상품링크' 최우선, 없으면 url/link/링크 패턴
    if "상품링크" in df.columns: 
        return "상품링크"
    for c in df.columns:
        if re.search(r"url|link|링크", str(c), flags=re.I):
            return c
    return None

def _normalize_df_list(df: pd.DataFrame) -> pd.DataFrame:
    col = _guess_url_col(df)
    if not col:
        raise ValueError("Step3 CSV에서 URL 컬럼을 찾을 수 없습니다(상품링크/url/link/링크 패턴).")
    if col != "상품링크":
        df = df.rename(columns={col: "상품링크"})
    # 링크 정리 + 중복 제거
    df = df.dropna(subset=["상품링크"]).copy()
    df["상품링크"] = df["상품링크"].astype(str).str.strip()
    df = df[df["상품링크"] != ""].drop_duplicates(subset=["상품링크"], keep="first")
    return df

def load_df_list_or_bootstrap() -> pd.DataFrame:
    # 1) 메모리에 이미 있으면 그대로
    if "df_list" in globals():
        d = globals()["df_list"]
        if isinstance(d, pd.DataFrame) and "상품링크" in d.columns and len(d) > 0:
            return d
        # 컬럼명이 다르면 정규화 시도
        if isinstance(d, pd.DataFrame):
            try:
                return _normalize_df_list(d)
            except Exception:
                pass

    # 2) 파일들에서 복원
    for path in _STEP3_SOURCES:
        if os.path.exists(path):
            try:
                df = pd.read_csv(path)
                df = _normalize_df_list(df)
                print(f"[Step4] 링크 목록 로드: {path} ({len(df)}행)")
                return df
            except Exception as e:
                print(f"[Step4] 목록 로드 실패({path}): {e}")

    raise FileNotFoundError(
        "Step3 산출물에서 링크 목록(df_list)을 복원할 수 없습니다.\n"
        "다음 중 하나를 제공해주세요:\n"
        " - step3_checkpoint/step3_ckeckpoint.csv\n"
        " - step3_products.csv"
    )


# === [ADD] 체크포인트 설정/유틸 ===

# ✅ 폴더/파일명 요구사항대로
STEP4_CKPT_DIR = "step4_checkpoint"
STEP4_CKPT_CSV = os.path.join(STEP4_CKPT_DIR, "step4_checkpoint.csv")
LAST_LOG = os.path.join(STEP4_CKPT_DIR, "step4_last_checkpoint.txt")

os.makedirs(STEP4_CKPT_DIR, exist_ok=True)

def _append_ckpt_row(row: dict):
    """1건씩 즉시 append 저장(헤더 자동)."""
    write_header = not os.path.exists(STEP4_CKPT_CSV)
    with open(STEP4_CKPT_CSV, "a", newline="", encoding="utf-8-sig") as f:
        writer = csv.DictWriter(f, fieldnames=list(row.keys()))
        if write_header:
            writer.writeheader()
        writer.writerow(row)

def _load_processed_set() -> set:
    """체크포인트 CSV 기반으로 '이미 처리한 URL' 집합 구성."""
    processed = set()
    if os.path.exists(STEP4_CKPT_CSV):
        try:
            _df = pd.read_csv(STEP4_CKPT_CSV)
            if "상품링크" in _df.columns:
                processed |= set(_df["상품링크"].dropna().astype(str).str.strip().tolist())
        except Exception:
            pass
    return processed


# ▷ 접속 제한(서비스 접속이 불가합니다) 감지 & 백오프 재시도
def _is_service_block_page(driver):
    try:
        html = (driver.page_source or "").lower()
    except Exception:
        html = ""
    # 대표 문구 / 버튼 텍스트 키워드
    keys = [
        "현재 서비스 접속이 불가합니다",  # 메인 경고
        "이전 페이지로 가기",
        "새로고침",
        "동시에 접속하는 이용자 수가 많거나",
    ]
    return any(k in html for k in keys)

def wait_if_service_block(driver, base_wait=30, max_wait=600, max_retries=3): # 최대 시도 횟수 3번
    import time, random
    wait_s = base_wait
    tries = 0
    if not _is_service_block_page(driver):
        return True  # 정상
    while _is_service_block_page(driver) and tries < max_retries:
        jitter = random.uniform(0.7, 1.3)
        sleep_for = min(int(wait_s * jitter), max_wait)
        print(f"⛔ 접속 제한 감지 → {sleep_for}초 대기 후 새로고침 시도 ({tries+1}/{max_retries})")
        time.sleep(sleep_for)
        # 새로고침 버튼 시도 → 실패 시 브라우저 refresh
        clicked = False
        try:
            # 버튼/링크 중 '새로고침' 텍스트를 가진 요소 클릭
            btns = driver.find_elements(By.XPATH, "//*[self::a or self::button][contains(., '새로고침')]")
            if btns:
                btns[0].click()
                clicked = True
        except Exception:
            pass
        if not clicked:
            try:
                driver.refresh()
            except Exception:
                pass
        tries += 1
        wait_s = min(int(wait_s * 1.8), max_wait)
    if _is_service_block_page(driver):
        print("⚠️ 접속 제한 해제 실패: 이 링크는 건너뜁니다.")
        return False
    return True


# ▷ 사람인지 확인(캡차 등) 감지 & 대기
def _is_human_check_page(driver):
    try:
        url = (driver.current_url or "").lower()
    except Exception:
        url = ""
    if any(k in url for k in ["captcha", "nidcaptcha", "/robot"]):
        return True
    try:
        html = (driver.page_source or "").lower()
    except Exception:
        html = ""
    # 대표적인 키워드(국문/영문 혼합)
    keywords = [
        "자동 접속 방지", "자동입력 방지", "자동 접속 차단", "보안문자", "보안 문자를 입력",
        "로봇이 아닙니다", "i am not a robot", "recaptcha", "hcaptcha"
    ]
    return any(k.lower() in html for k in keywords)

def wait_if_human_check(driver, prompt=True, poll_interval=2.0, max_wait_sec=7200):
    # 사람인지 확인 화면이면 사용자 입력을 기다리고, 해결될 때까지 주기적으로 체크
    import time
    if not _is_human_check_page(driver):
        return
    msg = "⚠️ 사람인지 확인 페이지 감지됨. 브라우저에서 인증을 완료하세요."
    if prompt:
        try:
            print(msg, "완료 후 Enter 키를 누르면 즉시 계속합니다.")
            input("인증 완료 후 Enter ▶ ")
            # Enter 이후에도 혹시 남아있으면 폴링으로 재확인
        except Exception:
            print(msg, "(입력 대기 불가 환경) 자동 감지로 전환합니다.")
    # 자동 감지 루프
    start = time.time()
    while _is_human_check_page(driver):
        if time.time() - start > max_wait_sec:
            print("오래 대기하여 다음 링크로 넘어갑니다.")
            break
        time.sleep(poll_interval)


# ▷ 스크롤: 화면 높이의 1.5배를 자연스럽게
def gentle_viewport_scroll(driver, times=1, factor=1.5):
    for _ in range(times):
        # vh = driver.execute_script("return window.innerHeight;")
        vh = driver.execute_script("return Math.max(window.innerHeight, document.documentElement.clientHeight);")
        dist = int(vh * factor)
        # 부드러운 스크롤(이미 smooth_scroll_by가 있다면 그걸 써도 OK)
        try:
            smooth_scroll_by(driver, dist, random.uniform(1.8, 3.2))
        except Exception:
            driver.execute_script(f"window.scrollBy(0,{dist});")
        time.sleep(random.uniform(1.2, 2.2))

# ▷ 표에서 key→value 맵 추출 (th-td 짝으로 읽기)
def table_to_dict(table_elem):
    kv = {}
    rows = table_elem.find_elements(By.CSS_SELECTOR, "tbody > tr")
    for tr in rows:
        ths = tr.find_elements(By.CSS_SELECTOR, "th")
        tds = tr.find_elements(By.CSS_SELECTOR, "td")
        # 2열/4열 구조를 모두 커버 (th-td, th-td)
        pairs = []
        # (th,td) 매칭 수 = min(len(ths), len(tds))
        for i in range(min(len(ths), len(tds))):
            key = ths[i].text.strip()
            val = tds[i].text.strip()
            if key:
                kv[key] = val
        # 일부 셀에 colspan이 있어도 위 방식으로 대부분 커버됨
    return kv

# ▷ 상세 페이지 파싱
def parse_detail_page(driver):
    """
    반환: dict
    채워야 할 필드:
      제조사, 브랜드, 모델명, 원산지, 두피타입, 모발타입, 타입,
      제품형태, 용량, 세부제품특징, 향계열, 종류, 성분, 주요제품특징
    """
    data = {
        "제조사":"",
        "브랜드":"",
        "모델명":"",
        "원산지":"",
        "두피타입":"",
        "모발타입":"",
        "타입":"",
        "제품형태":"",
        "용량":"",
        "세부제품특징":"",
        "향계열":"",
        "종류":"",
        "성분":"",
        "주요제품특징":""
    }

    # 1) 기본 스펙표(제조사/브랜드/모델명/원산지 등)
    #    예시 구조 확인: 상단 '상품정보' 표의 th-td (브랜드/원산지/제조사/모델명 등) 
    #    (div.BQJHG3qqZ4 내 table.RCLS1uAn0a 등)  :contentReference[oaicite:3]{index=3} :contentReference[oaicite:4]{index=4}
    base_tables = driver.find_elements(By.CSS_SELECTOR, "div.BQJHG3qqZ4 div.m_PTftTaj7 table")
    for tbl in base_tables:
        kv = table_to_dict(tbl)
        for k, v in kv.items():
            if k in ("제조사","브랜드","모델명","원산지"):
                data[k] = v

    # 2) 세부 속성표(두피타입/모발타입/타입/제품형태/용량/세부제품특징/향계열/종류/성분/주요제품특징)
    #    예시 구조 확인: div.detail_attributes 내 table.RCLS1uAn0a 의 th-td  :contentReference[oaicite:5]{index=5}
    detail_tables = driver.find_elements(By.CSS_SELECTOR, "div.detail_attributes div.m_PTftTaj7 table")
    for tbl in detail_tables:
        kv = table_to_dict(tbl)
        for key in ("두피타입","모발타입","타입","제품형태","용량","세부제품특징","향계열","종류","성분","주요제품특징"):
            if key in kv and not data[key]:
                data[key] = kv[key]

    # 3) 후처리: 괄호 단위 표기 정리 등 필요 시 여기에 추가
    #    예: ml(g) → ml 정규화, 쉼표/공백 트리밍 등
    return data

# ▷ 사람처럼 느리게 이동(다음 링크로 넘어가기 전 대기)
def human_delay(min_s=6.5, max_s=12.0):
    time.sleep(random.uniform(min_s, max_s))

# ▷ 판매중지(비판매) 페이지 감지
def is_suspended_product(driver):
    try:
        html = (driver.page_source or "").lower()
    except Exception:
        html = ""
    # 핵심 키워드만 사용 (품절은 제외)
    keys = [
        "판매중지",           # 일반 표기
        "판매 중지",          # 띄어쓰기 변형
        "판매중지 된 상품",    # 문장형
        "판매중지된 상품",
        "이 상품은 현재 판매중지"  # 안내문 일부
    ]
    return any(k in html for k in keys)

# ▷ '상품이 존재하지 않습니다' 페이지 감지
def is_no_product_page(driver):
    try:
        url = (driver.current_url or "").lower()
    except Exception:
        url = ""
    # 네이버 no-product 고정 경로가 종종 포함됨
    if "/no-product" in url:
        return True
    try:
        html = (driver.page_source or "").lower()
    except Exception:
        html = ""
    keys = [
        "상품이 존재하지 않습니다",   # 메인 문구
        "삭제되었거나",             # 보조 문구 일부
        "이전 페이지로 가기"         # 버튼 문구
    ]
    return any(k in html for k in keys)

# ▷ 특정 CSS들 중 "하나라도" 보이면 True (짧은 대기)
def wait_for_any_css(driver, selectors, timeout=8):
    import time
    end = time.time() + timeout
    while time.time() < end:
        for sel in selectors:
            try:
                if driver.find_elements(By.CSS_SELECTOR, sel):
                    return True
            except Exception:
                pass
        time.sleep(0.3)
    return False

# ▷ '첨부파일/비정형' 상세 감지 (표준 스펙 섹션이 없고, 첨부/파일 키워드만 보이는 경우)
def is_attachment_like_page(driver):
    try:
        html = (driver.page_source or "").lower()
    except Exception:
        html = ""
    # 자주 등장하는 키워드들 (필요시 추가)
    keys = [
        "첨부파일", "attachment", "파일 다운로드", "download file",
        "no image", "이미지가 없습니다"
    ]
    # 표준 스펙 섹터 유무
    has_spec = False
    try:
        has_spec = bool(driver.find_elements(By.CSS_SELECTOR, "div.BQJHG3qqZ4, div.detail_attributes"))
    except Exception:
        has_spec = False
    return (not has_spec) and any(k in html for k in keys)

def _log_short(idx:int, total:int, url:str, reason:str):
    # 예: [12/350] 상품없음 - https://smartstore.naver.com/...
    try:
        print(f"[{idx}/{total}] {reason} - {url}")
    except Exception:
        # url에 이상한 문자가 있어도 진행 막지 않기
        print(f"[{idx}/{total}] {reason}")



# ===== 본 처리: df_list['상품링크'] 순회 =====
df_list = load_df_list_or_bootstrap()  # ← df_list 자동 복원/정규화
wait = WebDriverWait(driver, 20)

# === [ADD] 재시작을 위한 완료 URL 집합 로드 ===
processed = _load_processed_set()
# 안전: 공백/NaN/타입 섞임 방지용 정규화
processed = {
    str(u).strip()
    for u in processed
    if u is not None and str(u).strip() != "" and str(u).lower() != "nan"
}
print("재시작용 완료 목록 로드:", len(processed), "건")

detail_rows = []
# 안전: NaN 제거 + 문자열화 + 공백 제거 + 빈문자 제거
links = [
    s for s in (
        df_list["상품링크"].dropna().astype(str).str.strip().tolist()
    ) if s
]

for idx, url in enumerate(links, start=1):
    if not url:
        continue
    if url in processed:
        print(f"[{idx}/{len(links)}] 이미 처리됨 → SKIP: {url}")
        continue

    print(f"[{idx}/{len(links)}] 이동: {url}")
    try:
        driver.get(url)
        # 링크당 안전 하드 타임아웃(전체 체류 시간 제한)
        LINK_HARD_TIMEOUT = 45  # 초
        link_start_ts = time.time()

        # 페이지 로드 직후, 표준 섹션이 없으면 짧게 기다렸다가 판단
        has_any_spec = wait_for_any_css(
            driver,
            selectors=["div.BQJHG3qqZ4", "div.detail_attributes"],
            timeout=8
        )

        # 상품 없음 / 판매중지 먼저 빠르게 처리
        if is_no_product_page(driver):
            row = {"상품링크": url, "제조사": "상품없음",
                "브랜드": None, "모델명": None, "원산지": None,
                "두피타입": None, "모발타입": None, "타입": None,
                "제품형태": None, "용량": None, "세부제품특징": None,
                "향계열": None, "종류": None, "성분": None, "주요제품특징": None}
            _append_ckpt_row(row); processed.add(url)
            try:
                with open(LAST_LOG, "w", encoding="utf-8") as _f:
                    _f.write(f"last_index={idx}\nprocessed_count={len(processed)}\nlast_url={url}\nlast_reason=상품없음\n")
            except Exception: pass
            _log_short(idx, len(links), url, "상품없음")
            human_delay(6.0, 12.0)
            continue

        if is_suspended_product(driver):
            row = {"상품링크": url, "제조사": "판매중지",
                "브랜드": None, "모델명": None, "원산지": None,
                "두피타입": None, "모발타입": None, "타입": None,
                "제품형태": None, "용량": None, "세부제품특징": None,
                "향계열": None, "종류": None, "성분": None, "주요제품특징": None}
            _append_ckpt_row(row); processed.add(url)
            try:
                with open(LAST_LOG, "w", encoding="utf-8") as _f:
                    _f.write(f"last_index={idx}\nprocessed_count={len(processed)}\nlast_url={url}\nlast_reason=판매중지\n")
            except Exception: pass
            _log_short(idx, len(links), url, "판매중지")
            human_delay(6.0, 12.0)
            continue

        # 첨부파일/비정형 상세 처리
        if (not has_any_spec) and is_attachment_like_page(driver):
            row = {"상품링크": url, "제조사": "첨부파일/비정형",
                "브랜드": None, "모델명": None, "원산지": None,
                "두피타입": None, "모발타입": None, "타입": None,
                "제품형태": None, "용량": None, "세부제품특징": None,
                "향계열": None, "종류": None, "성분": None, "주요제품특징": None}
            _append_ckpt_row(row); processed.add(url)
            try:
                with open(LAST_LOG, "w", encoding="utf-8") as _f:
                    _f.write(f"last_index={idx}\nprocessed_count={len(processed)}\nlast_url={url}\nlast_reason=첨부파일\n")
            except Exception: pass
            _log_short(idx, len(links), url, "첨부파일/비정형")
            human_delay(6.0, 12.0)
            continue




        # 캡차(사람 확인) 감지 시 잠시 정지 후 재개
        wait_if_human_check(driver)
        # 접속 제한 페이지면 백오프+새로고침으로 해제 시도
        ok = wait_if_service_block(driver, max_retries=3)  # ← 3회 재시도
        if not ok:
            # ✅ 안전 종료: 현재 진행 위치 로그 남기고 종료
            try:
                with open(LAST_LOG, "w", encoding="utf-8") as _f:
                    _f.write(f"last_index={idx}\n")
                    _f.write(f"processed_count={len(processed)}\n")
                    _f.write(f"last_url={url}\n")
                    _f.write("last_reason=service_block_exhausted\n")
            except Exception:
                pass
            raise SystemExit("접속 제한 지속 → 안전 종료 (다음 실행에서 재시작)")
    except Exception as e:
        print("  이동 실패:", e)
        human_delay(6.0, 12.0)
        continue
    
    # # ✅ 상품 없음 페이지 처리: 제조사='상품없음'으로 기록하고 다음으로
    # try:
    #     if is_no_product_page(driver):
    #         row = {
    #             "상품링크": url,
    #             "제조사": "상품없음",   # ← 요청사항
    #             "브랜드": None,
    #             "모델명": None,
    #             "원산지": None,
    #             "두피타입": None,
    #             "모발타입": None,
    #             "타입": None,
    #             "제품형태": None,
    #             "용량": None,
    #             "세부제품특징": None,
    #             "향계열": None,
    #             "종류": None,
    #             "성분": None,
    #             "주요제품특징": None,
    #         }
    #         _append_ckpt_row(row)        # 체크포인트 CSV에 즉시 append
    #         processed.add(url)           # 이번 세션 중복 방지

    #         # 진행 로그(선택)
    #         try:
    #             with open(LAST_LOG, "w", encoding="utf-8") as _f:
    #                 _f.write(f"last_index={idx}\n")
    #                 _f.write(f"processed_count={len(processed)}\n")
    #                 _f.write(f"last_url={url}\n")
    #                 _f.write("last_reason=상품없음\n")
    #         except Exception:
    #             pass

    #         human_delay(6.0, 12.0)
    #         continue
    # except Exception:
    #     # 감지 중 예외는 무시하고 일반 파싱으로 진행
    #     pass

    # # ✅ 판매중지 상품 처리: 제조사='판매중지'로 기록하고 스킵
    # try:
    #     if is_suspended_product(driver):
    #         row = {
    #             "상품링크": url,
    #             "제조사": "판매중지",   # 값이므로 문자열 유지
    #             "브랜드": None,
    #             "모델명": None,
    #             "원산지": None,
    #             "두피타입": None,
    #             "모발타입": None,
    #             "타입": None,
    #             "제품형태": None,
    #             "용량": None,
    #             "세부제품특징": None,
    #             "향계열": None,
    #             "종류": None,
    #             "성분": None,
    #             "주요제품특징": None,
    #         }
    #         # 체크포인트 즉시 저장 (step4_checkpoint 사용 버전)
    #         _append_ckpt_row(row)

    #         processed.add(url)
    #         try:
    #             with open(LAST_LOG, "w", encoding="utf-8") as _f:
    #                 _f.write(f"last_index={idx}\n")
    #                 _f.write(f"processed_count={len(processed)}\n")
    #                 _f.write(f"last_url={url}\n")
    #                 _f.write("last_reason=판매중지\n")
    #         except Exception:
    #             pass

    #         human_delay(6.0, 12.0)
    #         continue
    # except Exception:
    #     # 감지 중 예외는 무시하고 일반 파싱으로 진행
    #     pass



    # 주요 영역 대기(최대 LINK_HARD_TIMEOUT 내에서만 시도)
    try:
        ok2 = wait_if_service_block(driver, max_retries=3)
        if not ok2:
            human_delay(8.0, 12.0); continue

        # 섹션이 없으면 짧게 재시도하면서 하드 타임아웃 체크
        while True:
            if wait_for_any_css(driver, ["div.BQJHG3qqZ4", "div.detail_attributes"], timeout=4):
                break
            if time.time() - link_start_ts > LINK_HARD_TIMEOUT:
                # 표준 섹션이 끝내 안 뜨면 '비정형'으로 처리하고 넘어감
                row = {"상품링크": url, "제조사": "비정형/타임아웃",
                       "브랜드": None, "모델명": None, "원산지": None,
                       "두피타입": None, "모발타입": None, "타입": None,
                       "제품형태": None, "용량": None, "세부제품특징": None,
                       "향계열": None, "종류": None, "성분": None, "주요제품특징": None}
                _append_ckpt_row(row); processed.add(url)
                try:
                    with open(LAST_LOG, "w", encoding="utf-8") as _f:
                        _f.write(f"last_index={idx}\nprocessed_count={len(processed)}\nlast_url={url}\nlast_reason=비정형/타임아웃\n")
                except Exception: pass
                human_delay(6.0, 12.0)
                # 다음 링크로
                raise StopIteration  # 루프 바깥 except로 넘어가 continue
    except StopIteration:
        continue
    except Exception:
        # 그래도 못 찾으면 조금 기다렸다가 진행(파싱은 시도)
        time.sleep(2.0)


    # 첫 화면 안정화 대기 (초기 1–2초 정지)
    time.sleep(random.uniform(1.0, 2.0))

    # 1.5-2배 화면 스크롤(1~2회 정도)
    gentle_viewport_scroll(driver, times=random.randint(1,2), factor=random.uniform(1.5, 2.0))

    # 상세 페이지 파싱
    parsed = {}
    try:
        parsed = parse_detail_page(driver)
    except Exception as e:
        print("  파싱 오류:", e)
        parsed = {}

    # === [CHANGE] 결과 즉시 체크포인트 저장 (CSV append) ===
    row = {
        "상품링크": url,
        "제조사": parsed.get("제조사") or None,
        "브랜드": parsed.get("브랜드") or None,
        "모델명": parsed.get("모델명") or None,
        "원산지": parsed.get("원산지") or None,
        "두피타입": parsed.get("두피타입") or None,
        "모발타입": parsed.get("모발타입") or None,
        "타입": parsed.get("타입") or None,
        "제품형태": parsed.get("제품형태") or None,
        "용량": parsed.get("용량") or None,
        "세부제품특징": parsed.get("세부제품특징") or None,
        "향계열": parsed.get("향계열") or None,
        "종류": parsed.get("종류") or None,
        "성분": parsed.get("성분") or None,
        "주요제품특징": parsed.get("주요제품특징") or None,
    }
    # 메모리 보관(선택)
    detail_rows.append(row)

    # CSV 즉시 append → 중간 중단에도 누락 최소화
    # ✅ 즉시 체크포인트 CSV에 저장
    try:
        _append_ckpt_row(row)
    except Exception as _e:
        print("CSV 저장 중 오류(무시하고 진행):", _e)

    # 메모리 처리 집합 갱신(이번 세션 내 중복 방지)
    processed.add(url)

    # 마지막 체크포인트 로그(디버깅)
    try:
        with open(LAST_LOG, "w", encoding="utf-8") as _f:
            _f.write(f"last_index={idx}\n")
            _f.write(f"processed_count={len(processed)}\n")
            _f.write(f"last_url={url}\n")
    except Exception:
        pass

    # 다음 페이지로 넘어가기 전, 사람 같은 긴 대기 (페이지 차단 완화)
    human_delay(6.0, 12.0)

# ✅ 누적 체크포인트 CSV 기준으로 병합
if os.path.exists(STEP4_CKPT_CSV):
    df_detail = pd.read_csv(STEP4_CKPT_CSV)
    if "상품링크" in df_detail.columns:
        df_detail = df_detail.drop_duplicates(subset=["상품링크"], keep="last")
else:
    # 첫 실행에서만 메모리 캐시 사용(아직 파일이 없을 수 있음)
    df_detail = pd.DataFrame(detail_rows)

print("상세 수집 누적 행수:", len(df_detail))

재시작용 완료 목록 로드: 9847 건
[1/10095] 이미 처리됨 → SKIP: https://ader.naver.com/v1/9xKZUaGVHVRpl6-BM9xfmBIOt-3pzfei33bmDk0aJVAF_U_afl5pTgh1s2bJouZT-0ECFTnjJWo99Jk9zjoe5ebbwOtTKO6osbQcwd8MBtFw7fmMhnErB_RpnO7GDiMsdcARNF-NIaNEuQJoR4pyYQNxen6J4Qepp5BikV-X9QBePYYFGkUwEi3HbzcFUO5x-6qCFnzBzX2y5DA-RNFt6fTxldAs_VtZoIClGvAtHgsmpTwOYCiCObOCLUOabJ3w2atoXvSLW-vTfOZcJav2bKjz0ohC5eKimjo97gXqPh_Vu7q3iVKHOeo1eVDjpUrK7tCzRSij74s0BSTjuwZ5Qn15uTw6yn6KcpmkXU-JlGhnB3uFVqLsQQnFo89NoEeEbjdFmTlFPH9QPDz_Z4lDnugmnU6B1wkrvl_EP3905oB8RS4NbdoMo3Hksl8PzuJ_AYE-GoUpW1olC4mrsFmLtIL7L34TtnPBeyMAaxPK8LSoDh5GlB57sbBxYUB3Up3T?c=pc.nplusstore.npla&t=0
[2/10095] 이미 처리됨 → SKIP: https://ader.naver.com/v1/BKdxaZ_OPKeK4lK-ly5ZIiQMbmDhmCKsJetl6FnapUHIL0lqRRvcly2Wvz1YPffEqIKZRDjzG8R60OVtBFhePsYj1GrbyLfN4ivWauSwe0rBpKWOCFvqqWq-kaHtsgnMg-bCKtqYluO8Cb6ts8U3YTFUq_FLkgE_3M8EwpdQAlRqFt3ajjxp5oj_BEIZ0ZrqQ0vNovtc_DGi_wP9XTQ7u28iNns00Aj6LTlpY5M6SlkZ4UALU6meGlz-M4_hoUml3ML-fp6x-KN7fi4B_7OPvy-a6Fu73QYMgw1V6wyTzOIbY4JIGbCwbyi7PWw6c1PtJ3YzHoUHQS2cWohhC_

# 5. 데이터 병합
- step3 상품 목록과 step4 상세 페이지 데이터 병합

In [60]:
def normalize_empty_to_nan(df):
    # 문자열 빈칸("","  ") → NaN
    return df.applymap(
        lambda x: np.nan if (isinstance(x, str) and x.strip() == "") else x
    )

# 예: Step4 최종 병합 직전
if os.path.exists(STEP4_CKPT_CSV):
    df_detail = pd.read_csv(STEP4_CKPT_CSV, keep_default_na=True)  # 기본값이면 빈칸→NaN
else:
    df_detail = pd.DataFrame(detail_rows)

df_detail = normalize_empty_to_nan(df_detail)
df_list   = normalize_empty_to_nan(df_list)

C:\Users\kbjoo\AppData\Local\Temp\ipykernel_50720\4105358402.py:3: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  return df.applymap(


In [61]:
df_merged = pd.merge(df_list, df_detail, on="상품링크", how="left")
display(df_merged.head(5))

,스토어,제목,원래가격,현재가격,할인율,상품링크,제조사,브랜드,모델명,원산지,두피타입,모발타입,타입,제품형태,용량,세부제품특징,향계열,종류,성분,주요제품특징
0,바이브랩,"바이브랩 4주 솔루션 초록 탈모 샴푸 우디플로럴머스크, 500ml, 1개",32000.00,30900,0.03,https://ader.naver.com/v1/9xKZUaGVHVRpl6-BM9xf...,YEGREENA,바이브랩,리바이 솔루션 안티 헤어 로스 스칼프 샴푸 우디플로럴머스크향,국산,모든두피용,모든 모발용,"탈모샴푸, 두피샴푸, 일반샴푸",펌프형,500ml,NaN,기타향,NaN,NaN,"저자극, 탈모케어, 세정력"
1,바이브랩,"바이브랩 4주 솔루션 초록 탈모 샴푸 우디플로럴머스크, 500ml, 2개",62000.00,60900,0.01,https://ader.naver.com/v1/BKdxaZ_OPKeK4lK-ly5Z...,YEGREENA,바이브랩,리바이 솔루션 안티 헤어 로스 스칼프 샴푸 우디플로럴머스크향,국산,모든두피용,모든 모발용,"탈모샴푸, 두피샴푸, 일반샴푸",펌프형,500ml,NaN,기타향,NaN,NaN,"저자극, 탈모케어, 세정력"
2,Solep,두피진정 탈모샴푸300ml+100ml 앰플증정,35000.00,25400,0.27,https://ader.naver.com/v1/XqrZw161cHIqOpjmniEy...,제품별도표기,솔랩,NaN,국산,모든두피용,모든 모발용,"탈모샴푸, 두피샴푸",펌프형,NaN,"두피케어, 각질케어, 트러블케어, 유분제거",NaN,NaN,NaN,"탈모케어, 저자극, 세정력"
3,아모레퍼시픽몰 헤어바디,라보에이치 여름 쿨샴푸 지성 탈모 대용량 두피스케일링 400ml&400ml리필&180ml,44600.00,29800,0.33,https://ader.naver.com/v1/BKdxaZ_OPKeK4lK-ly5Z...,아모레퍼시픽,라보에이치,NaN,국산,"모든두피용, 지성",모든 모발용,"탈모샴푸, 두피샴푸, 쿨샴푸",펌프형,NaN,"상쾌함, 청량감, 실리콘 프리, 약산성, 유분제거",NaN,NaN,NaN,"탈모케어, 세정력, 저자극"
4,아모레퍼시픽몰 헤어바디,려 루트젠 두피 에센스 대용량 145ml 두피 영양제 여성 남성 탈모 앰플,36000.00,23900,0.33,https://smartstore.naver.com/main/products/115...,아모레퍼시픽,려,NaN,국산,모든두피용,모든 모발용,NaN,NaN,NaN,NaN,NaN,NaN,NaN,탈모케어


In [62]:
# ▷ 저장 (원하면 파일명/경로 조정)
today_date = datetime.now().strftime("%Y%m%d_%p%I%M%S")
out_csv = f"naver_hairloss_products_with_specs_{today_date}.csv"
df_merged.to_csv(out_csv, index=False, encoding="utf-8-sig")
print("저장:", out_csv)

저장: naver_hairloss_products_with_specs_20250929_PM114126.csv


In [63]:
# 드라이버 종료
driver.quit()